In [1]:
from numpy.ma.core import shape
import random

import numpy as np
import tensorflow as tf

from keras.datasets import mnist
from keras.datasets import fashion_mnist
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense 

import matplotlib.pyplot as plt 
from keras import Sequential
from keras.models import  model_from_json
import cv2

In [5]:




SEED_VALUE=42
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)

(x_trainall, y_trainall), (x_test, y_test) = mnist.load_data()

x_valid=x_trainall[:10000]
x_train=x_trainall[10000:]

y_Valid=y_trainall[:10000]
y_train=y_trainall[10000:]

# print(x_train,shape)
# print(x_valid,shape)
# print(x_test,shape)

# plt.figure(figsize=(15,5))
# for i in range(3):
#   plt.subplot(1,3,i+1)
#   # plt.axis()
#   plt.imshow(x_train[i],cmap='gray')
#   plt.subplots_adjust(0.2,0.2)

# plt.show()


x_train=x_train.reshape(x_train.shape[0],28*28)
x_train=x_train.astype('float32')/255

x_valid=x_valid.reshape(x_test.shape[0],28*28)
x_valid=x_valid.astype('float32')/255

x_test=x_test.reshape(x_test.shape[0],28*28)
x_test=x_test.astype('float32')/255

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
y_Valid=to_categorical(y_Valid)


model = Sequential()
model.add(Dense(128, activation='LeakyReLU', input_shape=(x_train.shape[1],)))
model.add(Dense(128, activation='LeakyReLU'))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# trainig_result= model.fit(x_train,y_train, batch_size=32, epochs=20, validation_data=(x_valid, y_Valid))

# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 128)               100480    
                                                                 
 dense_10 (Dense)            (None, 128)               16512     
                                                                 
 dense_11 (Dense)            (None, 10)                1290      
                                                                 
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
 

In [6]:
# load json and create model
json_file = open('updatemodel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("updatemodel.h5")
print("Loaded model from disk")

Loaded model from disk


In [7]:
def getpic(filename):
        img = cv2.imread(filename)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, threshold = cv2.threshold(gray, 110, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        rectangles = [cv2.boundingRect(c) for c in contours]
        sorted_rectangles = sorted(rectangles, key=lambda r: r[2]*r[3], reverse=True)
        # max_rect = max(rectangles, key=lambda r: r[2]*r[3])
        sorted_rectangles = sorted(rectangles, key=lambda r: r[2]*r[3], reverse=True)
        max_rects = sorted_rectangles[:2]
        cropped1=[]
        for i, rect in enumerate(max_rects):
          x, y, w, h = rect
          center_x = x + w/2
          center_y = y + h/2
          # cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
          # cv2.circle(img, (int(center_x), int(center_y)), 5, (0, 0, 255), -1)

          # برش  مستطیل
          cropped = img[y:y+h, x:x+w]
          
          cv2.imwrite(f'Imagederect{i}.png', cropped)
          cropped1.append(f'Imagederect{i}.png')
        
        return cropped1

In [8]:
# getpic('44.png')
images= getpic('75.png')
print(images)

['Imagederect0.png', 'Imagederect1.png']


In [9]:
def make_square(image_path, square_size=440):
    img = cv2.imread(image_path)
    height, width, _ = img.shape
    if height == width:
        return img
    elif height > width:
        top_bottom = height - width
        top = top_bottom // 2
        bottom = top_bottom - top
        img_with_border = cv2.copyMakeBorder(img, top, bottom, 0, 0, cv2.BORDER_CONSTANT, None, value=[0, 0, 0])
    else:
        left_right = width - height
        left = left_right // 2
        right = left_right - left
        img_with_border = cv2.copyMakeBorder(img, 0, 0, left, right, cv2.BORDER_CONSTANT, None, value=[0, 0, 0])
    resized_img = cv2.resize(img_with_border, (square_size, square_size))
    cv2.imwrite("resize.png",resized_img)
    return "resize.png"

In [10]:
def testmodel(filename):
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    img = img.astype('float32') / 255.0
    img = img.reshape(1, 28*28)
    pred = loaded_model.predict(img)
    print(np.argmax(pred))
    class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    pred_class_name = class_names[np.argmax(pred)]
    print('Predicted class:', pred_class_name)

In [11]:
from PIL import Image
import numpy as np
def testmodelupdate(filename):
          # Load the image and convert it to grayscale
          print(filename)
          image = Image.open(filename).convert("L")
          image = image.resize((28, 28))
          image_array = np.asarray(image)
          image_array = image_array.astype("float32") / 255.0
          image_array = np.reshape(image_array, (1, 28, 28, 1))
          predicted_output = loaded_model.predict(image_array)
          predicted_class = np.argmax(predicted_output)
          print("Predicted class:", predicted_class)

In [ ]:
resized_image = make_square("75.png")
print(resized_image)
testmodelupdate(resized_image)

# testmodel("resized_image1.png")

In [ ]:
from tkinter import *
root = Tk()
root.title('My First GUI...')
root.geometry('500x300')
root.configure(bg='red')
def hello():
    print("Hello ")
im = PhotoImage(file='./18.png')
im = im.subsample(2,2)
btn1 = Button(root,text='click    ',bg='yellow',fg='green',font=('arial',25,'bold'),activebackground='blue',
              width=200,height=100,anchor=N,cursor='clock',activeforeground='white',bd=8,relief=RIDGE,command=hello,
              image=im,compound=RIGHT)
btn1.pack()

lab1 = Label(root,text='Hello',anchor=N,bg='white',fg='black',
             font=('chiller',35,'italic bold underline'),cursor='watch',relief=RIDGE,bd=5,
             image=im,compound=BOTTOM)
lab1.pack()


root.mainloop()

In [ ]:
for i in range(len(images)) :
          print (i)
          resized_image = make_square(images[i])
          cv2.imwrite(f"resized_image{i}.png",resized_image)
          # testmodel(f"resized_image{i}.png")
          cv2.imshow('image', resized_image)
          cv2.waitKey(0)
          cv2.destroyAllWindows()

In [1]:
def addborder(filename):
          img = cv2.imread(filename)
          height, width, _ = img.shape
          border_size = 85
          border_color = (0, 0, 0)
          img_with_border = cv2.copyMakeBorder(img, border_size, border_size, border_size, border_size, cv2.BORDER_CONSTANT, value=border_color)
          cv2.imwrite(f"{filename}", img_with_border)
          return f"{filename}"

In [ ]:
# images= getpic('7.png')

img= addborder('./Imagederect0.png')
print(img)
# resized_image = make_square(img)
# print(resized_image)
# print(img)
# addborder('Imagederect1.png')
# testmodel('image_with_border.jpg')

In [ ]:
resized_image = make_square('resized_image0.png')
cv2.imwrite("resized_image.png",resized_image)
cv2.imshow('image', resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
testmodel('resized_image0.png')
testmodel('resized_image1.png')

In [ ]:


# تعیین مشخصات وب کم
camera_port = 0
ramp_frames = 30
camera = cv2.VideoCapture(camera_port)

# چک کردن اینکه وب کم را می‌توان دسترسی پیدا کرد یا نه
def get_image():
    # خواندن تعدادی فریم برای آماده سازی کیفیت وب کم
    for i in range(ramp_frames):
        retval, im = camera.read()
    # خواندن یک فریم واقعی از وب کم
    retval, im = camera.read()
    return im

while True:
    # خواندن یک فریم از وب کم
    ret, frame = camera.read()
    
    # نمایش فریم در پنجره
    cv2.imshow('Webcam Feed', frame)
    
    # اگر کاربر کلید q را فشار داد
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # خروج از حلقه بی نهایت و آزاد کردن دوربین
        camera.release()
        cv2.destroyAllWindows()
        break
    
    # اگر کاربر کلید s را فشار داد
    elif cv2.waitKey(1) & 0xFF == ord('s'):
        image= getpic(frame=frame)
        resized_image=make_square
        testmodel(resized_image)
        camera.release()
        cv2.destroyAllWindows()
        break